In [2]:
import os
import numpy as np
import pandas as pd

Read in the deal data and the URLs.

In [1]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [3]:
path = find('dealdata.xlsx','C://')
transactions = pd.read_excel(path)

In [4]:
path = find('CompanyUrls.xlsx', 'C://')
urls = pd.read_excel(path)

In [5]:
transactions = transactions.reset_index(drop=True)

Count number of deals

In [6]:
print('Between 1-1-2010 and 31-12-2017, ' + str(transactions.shape[0]) + ' M&As were completed in Hospital and Medical Service Plans and Accident and Health Insurance')
print('These deals involved '+ str(transactions['Target Name'].nunique()) + ' targets and ' + str(transactions['Acquiror Name'].nunique())
    + ' acquirors')

Between 1-1-2010 and 31-12-2017, 708 M&As were completed in Hospital and Medical Service Plans and Accident and Health Insurance
These deals involved 658 targets and 453 acquirors


Find for each transaction the corresponding URLs and dates from which the data collection was started before/after the deal

In [7]:
url_data = pd.DataFrame()
for index, row in transactions.iterrows():
    target = row['Target Name']
    acquiror = row['Acquiror Name']
    date = row['Date Effective'].strftime('%Y-%m-%d')
    corresponding_url = urls[(urls['Target Name']==target)&(urls['Acquiror Name']==acquiror)&(urls['Date Effective']==date)]
    if corresponding_url.shape[0]>1: 
        corresponding_url=corresponding_url.iloc[0]
    url_data = url_data.append(corresponding_url)    

In [8]:
transactions = transactions.reset_index(drop=True)
url_data = url_data.reset_index(drop=True)

In [9]:
url_data = url_data.drop(['Target Name', 'Acquiror Name', 'Date Effective'], axis=1)

Concatenate the two dataframes

In [10]:
trans_url= pd.concat([transactions, url_data], axis=1)

In [11]:
for index, row in trans_url.iterrows():
    if (row['Target URL']=='None'):
        trans_url.at[index, 'Target URL'] = None
    if (row['Acquiror URL']=='None'):
        trans_url.at[index, 'Acquiror URL'] = None

In [12]:
print(str(trans_url['Target URL'].count()) + ' Target URLs were found. '+ str(trans_url['Target URL'].nunique()) + ' of those were unique.')
print(str(trans_url['Acquiror URL'].count()) + ' Acquiror URLs were found. ' + str(trans_url['Acquiror URL'].nunique()) + ' of those were unique.')


557 Target URLs were found. 515 of those were unique.
584 Acquiror URLs were found. 379 of those were unique.


Save the dataframe

In [13]:
trans_url.to_excel('C:/TransAndUrl.xlsx')